In [16]:
# insert_symbols.py
import datetime
from math import ceil
import bs4
import MySQLdb as mdb
import requests

In [4]:

def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P500
    constituents using requests and BeautifulSoup.
    Returns a list of tuples for to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.now(datetime.UTC)
    # Use requests and BeautifulSoup to download the
    # list of S&P500 companies and obtain the symbol table
    response = requests.get(
    "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text)
    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]
    # Obtain the symbol information for each
    # row in the S&P500 constituent table
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select('td')
        symbols.append(
            (
            tds[0].select('a')[0].text, # Ticker
            'stock',
            tds[1].select('a')[0].text, # Name
            tds[3].text, # Sector
            'USD', now, now
            )
        )
    return symbols

sp = obtain_parse_wiki_snp500()
sp

[('MMM',
  'stock',
  '3M',
  'Industrial Conglomerates',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('AOS',
  'stock',
  'A. O. Smith',
  'Building Products',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('ABT',
  'stock',
  'Abbott',
  'Health Care Equipment',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('ABBV',
  'stock',
  'AbbVie',
  'Biotechnology',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('ACN',
  'stock',
  'Accenture',
  'IT Consulting & Other Services',
  'USD',
  d

In [23]:
def insert_snp500_symbols(symbols):
    """
    Insert the S&P500 symbols into the MySQL database.
    """
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'home_user'
    db_pass = '787753'
    db_name = 'securities_master'
    con = mdb.connect(
    host=db_host, user=db_user, passwd=db_pass, db=db_name
    )
    # Create the insert strings
    column_str = (
    "ticker, instrument, name, sector, "
    "currency, created_date, last_updated_date"
    )
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
    (column_str, insert_str)
    # Using the MySQL connection, carry out
    # an INSERT INTO for every symbol
    cur = con.cursor()
    cur.executemany(final_str, symbols)
    con.commit()
    cur.close()
    con.close()
    # if __name__ == "__main__":
    #     symbols = obtain_parse_wiki_snp500()
    #     insert_snp500_symbols(symbols)
    #     print("%s symbols were successfully added." % len(symbols))


In [24]:
sp = obtain_parse_wiki_snp500()
insert_snp500_symbols(sp)

In [10]:
#Establish connection 
 # Connect to the MySQL instance
db_host = 'localhost'
db_user = 'home_user'
db_pass = '787753'
db_name = 'securities_master'
con = mdb.connect(
host=db_host, user=db_user, passwd=db_pass, db=db_name
)

cursor = con.cursor()

#id connections
cursor.execute("SELECT id from information_schema.processlist WHERE user = 'home_user'")
connections = cursor.fetchall()

#Terminate connections
for connection in connections: 
    kill_command = f"KILL {connection[0]}"
    cursor.execute(kill_command)

#Close cursor and connections 
cursor.close()
con.close()

OperationalError: (1317, 'Query execution was interrupted')

In [22]:
print(os.getenv('SQL_PASSWORD'))

None
